# Collaboration

There are language features in Python to help you construct well-defined APIs with clear interface boundaries. The Python community has established best practices that maximize the maintainability of code over time. There are also standard tools that ship with Python that enable large teams to work together across disparate environments.

## Item 49: Write Docstrings for Every Function, Class, and Module

Documentation in Python is extremely important because of the dynamic nature of the language. Python provides built-in support for attaching doc to blocks of code.

The documentation from a program's source code is directly accessible as the program runs. Docstrings can be attached to functions, classes, and modules. This connection is part of the process of compiling and running a Python program.

In [2]:
def palindrome(word):
    """Return True if the given word is a palindrome."""
    return word == word[::-1]

print(repr(palindrome.__doc__))

'Return True if the given word is a palindrome.'


Support for docstrings and the `__doc__` attribute has three consequences:
* The accessibility of doc makes interactive dev easier. (by `help` built-in function)
* A standard way of defining doc makes it easy to build tools that convert the text into more appealing formats (like HTML). ([Sphinx](http://sphinx-doc.org),[Read the Docs](https://readthedocs.org))
* Python's first-class, accessible, and good-looking doc encourage people to write more doc.

You need to follow [PEP 257](https://www.python.org/dev/peps/pep-0257/).

### Documenting Modules

Each module should have a top-level docstring. Use """. The first line of the docstring should be a single sentence describing the module's purpose.

```python
# words.py
#!/usr/bin/env python3
"""Library for testing words for various lingustic patterns.

Test how words relate to each other can be tricky sometimes.
This module ...

Available functions:
- palindrome: Determine if a word is a palindrome.
- check_anagram: Determine if two words are anagrmas.
...
"""
```

### Documenting Classes

Each class should have a class-level docstring. This largely follows the same pattern as the module-level docstring.

Important public attributes and methods of the class should be highlighted in the class-level docstring.

```python
class Player(object):
    """Represents a player of the game.
    
    Subclasses may override the 'tick' method to provide custom animations for the play's movement depending on their power level, etc.
    
    Public attributes:
    - power: Unused power-ups (float between 0 and 1)
    - coins: Coins found during the level (integer)
    """
    
    #...
```

### Documenting Functions

Each **public** function and method should have a docstring. This follows the same pattern as modules and classes.

```python
def find_anagrams(word, dictionary):
    """Find all anagrams for a word.
    
    This function only runs as fast as the test for membership in the 'dictionary' container.
    It will be slow if the dictionary is a list and fast if it's a set.
    
    Args:
        word: String of the target word.
        dictionary: Container with all strings that are known to be actual words.
        
    Returns:
        List of anagrams that were found. Empty if none were found.
    """
    
    # ...
```

## Item 50: Use Packages to Organize Modules and Provide Stable APIs

As the size of a program's codebase grows, it's natural for you to reorganize its structure. You split larger funcs into **smaller funcs**. You refactor data structures into **helper classes**. Then maybe you seprarate functionality into various **modules** that depend on each other.

At some point, you may find yourself with so many modules that you need another layer in your program to make it understandable. Python provides **packages**, **packages are modules** that contain other modules.

In most cases, packages are defined by putting an empty file named `__init__.py` into a directory. Once `__init__.py` is present, any other Python files in that directory will be available for import using a path relative to the directory. Imagine that you have the following dir struct:

```
main.py
mypackage/__init__.py
mypackage/models.py
mypackage/utils.py
```

Use the following code to import `utils`:

```
# in main.py
from mypackage import utils
```

This pattern continues when you have package dir present within other packages.

PS: Python 3.4 introduces **namespace packages**.

The functionality provided by packages has two primary purposes in Python programs.

* Namespaces: when names conflict; use `as`;
* Stable APIs: `__all__` special attribute;

```python
# __init__.py
__all__ = []
from . models import *
__all__ += models.__all__
from . utils import *
__all__ += utils.__all__
```


## Item 51: Define a Root Exception to Insulate Callers from APIs


## Item 52: Know How to Break Circular Dependencies


## Item 53: Use Virtual Environments for Isolated and Reproducible Dependencies

Building larger and more complex programs often leads you to rely on various packages from the Python community. You'll find yourself running pip to install packages like `pytz`, `numpy` and many others.

The problem is that, by default, `pip` installs new packages in a global location. That causes all Python programs on your system to be affected.

**The trouble comes from transitive dependencies: the packages that the packages you install depend on**.

Use the command to show dependencies:

```shell
$ pip show Sphinx
```

Python can only have a single global version of a module, if Sphinx and flash both depend on `Jinja2`, and we update `Jinja2`, then some breakage would happen in some time.

These difficulties are magnified when you colaborate with other programmers who do their work on separate computer. In .NET for third-party packages, we normally use nuget and a xml config file to specify dependencies specific to the current project, it works fine. 

The similar solution to these problems is a tool called `pyvenv`, which provides **virtual environments**.

`pyvenv` allows you to create isolated versions of the Python environment. Using pyvenv, you can have many different versions of the same package installed on the same system at the same time without conflicts. This lets you work on many different projects and use many different tools on the same computer.

```shell
$ /usr/local/bin/pyvenv ~/works/py/myproj
$ cd ~/works/py/myproj
$ ls
bin  include  lib  pyvenv.cfg
```

To start the virtual env, we can use the `source` cmd from shell on the `bin/activate`

```shell
$ source bin/activate
(myproj)$

(myproj)$ which python3
.../myproj/bin/python3
```

This ensures that changes to the outside system will not affect the virtual env. The virtual env we just created with `pyvenv` starts with no packages installed except for `pip` and `setuptools`. You can verify it with:

```shell
$ python3 -c 'import pytz'
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ImportError: No module named 'pytz'
```

```shell
$ python3 -c 'import pytz'
```

We can use `deactivate` cmd to restore the env of the system defaults.

### Reproducing Dependencies

Once you have a virtual environment, you can continue installing packages with pip as you need them. Eventually, you may want to copy your environment somewhere else. For example, say you want to **reproduce your development environment on a production server**. Or maybe you want to **clone someone else’s environment on your own machine** so you can run their code.

`pyvenv` makes these situations easy. You can use the `pip freeze` command to save all of your explicit package dependencies into a file. By convention, this file is named **requirements.txt**.

```shell
$ pip3 freeze > requirements.txt
$ cat requirements.txt
numpy==1.11.0
pytz==2016.4
```

Then create a new env named 'newproj', check installed packages and install other via requirements.txt:

```shell
$ pip3 list
pip (7.1.2)
setuptools (18.2)

$ pip3 install -r ../myproj/requirements.txt
```

The gotchar with virtual environments is that moving them breaks everything because all of the paths, like `python3`, are hard-coded to the env's install dir.

In versions of Python before 3.4, the `pyvenv` tool must be downloaded and installed separately, the command-line tool is called `virtualenv`.